In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Libarary import

import numpy as np                  # data arrays
import pandas as pd                 # data reading
from tqdm import tqdm               # processing percentage bar
from lightgbm import LGBMRegressor  # gradient boosting regressor

# Data reading

data = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')

# Data handling

    ## Feature distribution

features = list(data.columns)         
features_1, features_2, features_3, features_4 = [], [], [], []
F = [[], [], [], [], []]
for feature in features:                     # Iterating features
    for i in [1, 2, 3, 4]:                   # Iterating data subsets
        if feature.split('_')[1] == str(i):  # Split a string for every underscore present 'F_4_11' = 3 splits
            F[i].append(feature)             # Appending feature to array
df = [[], [], [], [], []]

for i in [1, 2, 3, 4]:
    df[i] = data[F[i]]  # Iterating data subsets
    
    ## Plot the number of null values per sample

features = list(data.columns)
missing_values_count = pd.DataFrame(data[features].isnull().sum())
missing_values_count.reset_index()
missing_values_count.plot()

# Parameters

parameters = {'random_state': 22, 'n_estimators': 5000, 'learning_rate' : 0.1}     
    # random_state = sets a seed to to the random generator, meaning the same results will be produced as long as the random state is consistent
    # n_splits = >10000, reduce to minimize runtime. n_estimators = number of trees or number of samples on which the algoritm will work
    # learning_rate = defines step_size at each iteration or how fast the model learns from the training set. 
    # High learning rates can cause model to learn too quickly and arrive at a non-optimal set of parameters 

# Modelling

for i in [1,3,4]:
    dataframe = pd.DataFrame()     # DataFrame = a 2d data structure, with this we can perform basic operations on rows/columns
    train_column = pd.DataFrame()
    test_column = pd.DataFrame()
    dataframe=df[i].copy()
    for column in dataframe.columns: 
        if dataframe[column].isnull().sum() == 0:
            continue                                                 # continue if no null values are present
        indexed_null = dataframe[dataframe[column].isnull()].index   # indexing rows with null values
        
        train_column = dataframe.drop(indexed_null, axis = 0)        # tranining set: at least one column has no null values  
        test_column = dataframe[dataframe.index.isin(indexed_null)]  
        
        X = train_column.drop([column],axis=1)
        y = train_column[column]
        
        model = LGBMRegressor(**parameters)
        model.fit(X,y)
        
        dataframe[column][indexed_null] = model.predict(test_column.drop([column],axis=1))
    df[i]=dataframe.copy()                                         
          
# Submission
    
merged_data = pd.concat([df[1], df[2], df[3], df[4]], axis=1)

for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = merged_data.loc[row, col]

submission.to_csv('submission.csv')

print(submission)